In [11]:
from typing import List, Tuple
from typing_extensions import Annotated

from langchain_core.documents import Document
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.tools import tool

from langgraph.prebuilt import InjectedState


@tool(parse_docstring=True, response_format="content_and_artifact")
def get_context(question: List[str]) -> Tuple[str, List[Document]]:
    """Get context on the question.

    Args:
        question: The user question
    """
    # return constant dummy output
    docs = [
        Document(
            "FooBar company just raised 1 Billion dollars!",
            metadata={"source": "twitter"},
        ),
        Document(
            "FooBar company is now only hiring AI's", metadata={"source": "twitter"}
        ),
        Document(
            "FooBar company was founded in 2019", metadata={"source": "wikipedia"}
        ),
        Document(
            "FooBar company makes friendly robots", metadata={"source": "wikipedia"}
        ),
    ]
    return "\n\n".join(doc.page_content for doc in docs), docs


@tool(parse_docstring=True, response_format="content_and_artifact")
def cite_context_sources(
    claim: str, state: Annotated[dict, InjectedState]
) -> Tuple[str, List[Document]]:
    """Cite which source a claim was based on.

    Args:
        claim: The claim that was made.
    """
    docs = []
    # We get the potentially cited docs from past ToolMessages in our state.
    for msg in state["messages"]:
        if isinstance(msg, ToolMessage) and msg.name == "get_context":
            docs.extend(msg.artifact)

    class Cite(BaseModel):
        """Return the index(es) of the documents that justify the claim"""

        indexes: List[int]

    structured_model = model.with_structured_output(Cite)
    system = f"Which of the following documents best justifies the claim:\n\n{claim}"
    context = "\n\n".join(
        f"Document {i}:\n" + doc.page_content for i, doc in enumerate(docs)
    )
    citation = structured_model.invoke([("system", system), ("human", context)])
    cited_docs = [docs[i] for i in citation.indexes]
    sources = ", ".join(doc.metadata["source"] for doc in cited_docs)
    return sources, cited_docs

In [12]:
import operator
from typing import Annotated, Sequence, TypedDict

from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [13]:
from copy import deepcopy

from langchain_core.messages import ToolMessage
from langchain_openai import ChatOpenAI

from langgraph.prebuilt import ToolNode

model = ChatOpenAI(model="gpt-4o", temperature=0)


# Define the function that determines whether to continue or not
def should_continue(state, config):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"


tools = [get_context, cite_context_sources]

# Define the function that calls the model
def call_model(state, config):
    messages = state["messages"]
    model_with_tools = model.bind_tools(tools)
    response = model_with_tools.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# ToolNode will automatically take care of injecting state into tools
tool_node = ToolNode(tools)

In [14]:
from langgraph.graph import END, START, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("action", "agent")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [15]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage("what's the latest news about FooBar")]
for output in app.stream({"messages": messages}):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
        messages.extend(value["messages"])
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_fjVgtlkC4uIYgmtWDiRC4PLJ', 'function': {'arguments': '{"question":["what\'s the latest news about FooBar"]}', 'name': 'get_context'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 87, 'total_tokens': 109}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3aa7262c27', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b79ed8f3-f4cd-407c-9595-618b34561cf6-0', tool_calls=[{'name': 'get_context', 'args': {'question': ["what's the latest news about FooBar"]}, 'id': 'call_fjVgtlkC4uIYgmtWDiRC4PLJ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 87, 'output_tokens': 22, 'total_tokens': 109})]}

---

Output from node 'action':
---
{'messages': [ToolMessage(content="FooBar company just raised 1 Billion dollars!\n\nFooBar company is now only hiring AI's\n\nFooBar company was founded in 2019\n\nFooBar comp

In [23]:
from langgraph_sdk import get_client

client = get_client(url="https://testing-imports-langgraph-r-a12e17b191b0592c8e5b9c432b7b8af7.default.us.langgraph.app")

In [24]:
assistants = await client.assistants.search()
assistant = assistants[0]

In [1]:
from langchain_experimental.openai_assistant import OpenAIAssistantRunnable
from langchain.agents import AgentExecutor
#from langchain.tools import E2BDataAnalysisTool


#tools = [E2BDataAnalysisTool(api_key="...")]
tools=[]
agent = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant e2b tool",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=tools,
    model="gpt-4-1106-preview",
    as_agent=True
)

agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke({"content": "What's 10 - 4 raised to the 2.7"})

TypeError: Assistants.create() got an unexpected keyword argument 'file_ids'

In [3]:
from langgraph.checkpoint_sqlite.aio import AsyncSqliteSaver

ModuleNotFoundError: No module named 'langgraph.checkpoint_sqlite'